In [37]:
import numpy as np
from tkinter import *
from functools import partial
import time
from datetime import datetime,timedelta
import random
import json
from playsound import playsound
import pandas as pd
from tkinter import filedialog



#qstns = {1:['Arrange in the decreasing order of area:',[('A. Rectangle with sides 1m and 0.5m',0.5),('B. Square with side 1m',1),('C. Right angled triangle with base 3m and height 2m',3),('D. Circle with radius 1m',3.14)]],2:['Arrange in ascending order']}
qstns = {1:['You have not entered the questions, add through Qstn path option in file menu',[['','']*4] ]}

global b
b = 1
global j
j = 0

def timer():
    global watchtime
    try:
        watchtime = datetime(1,1,1,0,mint,secc)
    except:
        watchtime = datetime(1,1,1,0,1,0)
    update()    
    
    
def update():
    global watchtime
    now = watchtime.strftime('%M:%S')
    ftmr.configure(text=now)
    try:
        watchtime = watchtime - timedelta(seconds = 1)
        fcomp.after(1000,update)
    except :
        ftmr.config(text='Time up!')

            
def timopt():
    setopt()
    timer()

def setqstn():
    global curr
    curr = qstns[str(b)]
    global sortcurr
    sortcurr = iter(sorted(curr[1], reverse = (b%2==1), key = lambda x: x[1]))
    fqstn.config(text=curr[0])
    
def setopt():
    for i in range(4):
        anss[i].config(text = curr[1][i][0])
        

def sortans():
    global j
    global b
    try:
        resp = next(sortcurr)
        anss[j].config(text= resp[0] + ' = '+str(resp[1]))
        if j<3:
            j += 1
        else:
            j=0
            b += 1
        
        #fscr.after(2000,sortans)
    except StopIteration:
        pass

        

def clr():
    for i in range(4):
        anss[i].config(text='')
        
    
def leaderboard():
    fqstn.place_forget()
    ldrbrd.place(relx = 0.05, rely = 0.1, relwidth = 0.6, relheight =0.8)
    df = pd.read_excel('C:/Users/Yannique/Downloads/apoll.xlsx', parse_dates=[0], usecols=[0,2,3])
    df.dropna(how='all',inplace=True)
    df['Time']=pd.to_datetime(df['Timestamp']-df.iloc[0,0]).dt.strftime('%M:%S.%f')
    for i in range(1,len(df)):
        plnam[i].config(text =df.iloc[i,1].upper())
        plans[i].config(text=df.iloc[i,2].upper())
        pltime[i].config(text=df.iloc[i,3][:8])
        
        
def revcor(event):
    for i in range(10):
        if plans[i]['text'] == 'DABC':
            plnam[i].config(bg='green')
            plans[i].config(bg='green')
            pltime[i].config(bg='green')

def submittime():
    global mint
    global secc
    mint = int(timm.get())
    secc = int(tims.get())
    tm.destroy()
    
    
def settime():
    global tm
    tm = Toplevel(fcomp)
    tm.geometry('250x120')
    l11 = Label(tm, text = 'Minutes')
    l12 = Label(tm,text = 'Seconds')
    e1 = Spinbox(tm, from_ = 0, to = 60, textvariable = timm)
    e2 = Spinbox(tm, from_=0, to = 59, textvariable = tims)
    ok = Button(tm,text='OK', command = submittime)
    l11.grid(row=0, column=0)
    l12.grid(row=1, column = 0)
    e1.grid(row=0, column=1)
    e2.grid(row=1,column=1)
    ok.grid(row=3, columnspan=2)
    
    

fcomp = Tk()
fcomp.title("Fastest Finger First")
fcomp.geometry('900x720')
fcomp.config(bg = 'blue')
fscr = Frame(fcomp, bg = 'black')
fscr.place(relx = 0.05, rely = 0.1, relwidth = 0.6, relheight =0.8)

ldrbrd = Frame(fcomp, bg = 'black')

def qstnfile():
    file = filedialog.askopenfile(mode='r', filetypes=[('Text files','*.txt')])
    if file is not None:
        global cont
        cont = file.readlines()
        print(len(cont))
        global qstns
        qstns= json.loads(cont[0])
        
################################################################33
topmen = Menu(fcomp, tearoff=0)

filemenu = Menu(topmen)
#filemenu.add_command('Result Path')
filemenu.add_command(label='Qstn Path', command = qstnfile)
newmenu = Menu(topmen)
#newmenu.add_command()

topmen.add_cascade(label='File',menu=filemenu)
topmen.add_command(label='Time', command=settime)
##############################################################

plnam = [Label(ldrbrd, fg='white',bg='black',font = ('Times New Roman', 15), anchor='w') for i in range(10)]
plans = [Label(ldrbrd, fg='white',bg='black',font = ('Times New Roman', 15), anchor='w') for i in range(10)]
pltime = [Label(ldrbrd, fg='white',bg='black',font = ('Times New Roman', 15), anchor='w') for i in range(10)]

for i in range(10):
    plnam[i].place(relx=0, rely=0+0.1*i,relwidth=0.4,relheight=0.1)
    plans[i].place(relx=0.4, rely=0+0.1*i,relwidth=0.3,relheight=0.1)
    pltime[i].place(relx=0.7, rely=0+0.1*i,relwidth=0.3,relheight=0.1)    
    

plnam[0].config(text= 'Player')
plans[0].config(text='Answer')
pltime[0].config(text='Time')


fqstn = Label(fscr , bg = 'blue', bd =5, fg='white',anchor = 'w', font = ('Times New Roman', 18), wraplength = 500, justify = LEFT)
fqstn.place(relx = 0.05, rely = 0.02, relwidth = 0.9, relheight = 0.23)

fcomp.bind('<Up>',revcor)

anss = [0,0,0,0]
for i in range(4):
    anss[i] = Label(fscr, bg ='blue',font = ('Times New Roman', 15),anchor='w',fg = 'white')
    anss[i].place(relx = 0.05, rely = 0.28 + i*0.18, relwidth = 0.9, relheight = 0.16)
    
    
ftmr = Label(fcomp)    
ftmr.place(relx = 0.72, rely = 0.1, relwidth = 0.2, relheight = 0.1)

butfr = Frame(fcomp)
butfr.place(relx = 0.7, rely = 0.4, relwidth = 0.25, relheight = 0.5)

fbuts = [0]*5
commands = [('Qstn',setqstn),('Options',timopt),('Clear',clr),('Sort',sortans),('Leaderboard', leaderboard)]
for i in range(5):
    fbuts[i] = Button(butfr, text = commands[i][0], command = commands[i][1])
    fbuts[i].pack(expand=True, fill=BOTH)
    
    
timm = StringVar()
tims = StringVar()

fcomp['menu'] = topmen
fcomp.mainloop()

1


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\Yannique\Anaconda3\lib\tkinter\__init__.py", line 1705, in __call__
    return self.func(*args)
  File "<ipython-input-37-cf8c8398d72c>", line 67, in sortans
    b += 1
UnboundLocalError: local variable 'b' referenced before assignment


In [9]:
b = 0

In [7]:
len(cont)

NameError: name 'cont' is not defined

In [35]:
curr

{'1': ['a',
  [['A. o1', '11'], ['B. o2', '22'], ['C. o3', '33'], ['D. o4', '44']]]}

In [36]:
qstns[0]

KeyError: 0

In [20]:
pwd

'C:\\Users\\Yannique'

In [30]:
fileref = open('kbggff.txt','r')

In [31]:
a=fileref.read()

In [32]:
a

'{"1": ["a", [["A. o1", "2"], ["B. o2", "3"], ["C. o3", "4"], ["D. o4", "6"]]], "2": ["a", [["A. o1", "6"], ["B. o3", "8"], ["C. o4", "7"], ["D. o2", "3"]]], "3": ["a", [["A. o3", "5"], ["B. o2", "6"], ["C. o4", "8"], ["D. o1", "4"]]]}'

In [33]:
json.loads(a)

{'1': ['a', [['A. o1', '2'], ['B. o2', '3'], ['C. o3', '4'], ['D. o4', '6']]],
 '2': ['a', [['A. o1', '6'], ['B. o3', '8'], ['C. o4', '7'], ['D. o2', '3']]],
 '3': ['a', [['A. o3', '5'], ['B. o2', '6'], ['C. o4', '8'], ['D. o1', '4']]]}